In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103

## 1. Análisis de Procedimientos a Nivel País

En este análisis se quieren responder las siguiente preguntas:

1. ¿Cómo es el ranking de los hospitales para cada uno de los procedimientos realizads?
2. ¿Quién es el Hospital que realiza más de X procedimientos?
3. ¿El procedimiento X es realizado en todo el país? ¿En qué partes en específico se realiza?

Para responder esta pregunta se utilizará la base de datos de Egresos de Hospitales adscritos
a GRD entre 2019 y 2021.


In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)
df_procesada["ANIO_EGRESO"] = df_procesada["ANIO_EGRESO"].astype("Int32")

In [5]:
display(df_procesada.head(10))

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA
0,118100,1314867.0,MUJER,1988-06-19,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,1.0,146102.0,0.5744,2.0,1.0,NaN,5.0,2019,3.0,2019-3
1,118100,1354418.0,MUJER,1988-03-27,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,NaN,134161.0,0.2951,1.0,1.0,NaN,4.0,2019,3.0,2019-3
2,118100,239861.0,MUJER,2017-06-17,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,64182.0,0.6736,2.0,2.0,NaN,6.0,2019,3.0,2019-3
3,118100,330326.0,HOMBRE,2017-12-04,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,NaN,64172.0,0.5475,2.0,2.0,NaN,2.0,2019,3.0,2019-3
4,118100,1369293.0,MUJER,1995-12-06,OTRO,CONCEPCION,FLORIDA,CHILE,FONASA INSTITUCIONAL - (MAI) C,CONCEPCIÓN,...,NaN,134141.0,0.3107,1.0,1.0,NaN,6.0,2019,3.0,2019-3
5,118100,469385.0,MUJER,1942-06-29,OTRO,CONCEPCION,CHIGUAYANTE,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,114103.0,1.1799,3.0,3.0,NaN,47.0,2019,3.0,2019-3
6,118100,1146635.0,MUJER,1951-07-14,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,44163.0,1.0328,3.0,3.0,NaN,43.0,2019,3.0,2019-3
7,118100,93557.0,HOMBRE,1938-03-04,OTRO,CONCEPCION,LOTA,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,1.0,11101.0,1.8731,1.0,1.0,NaN,41.0,2019,3.0,2019-3
8,118100,674943.0,MUJER,1962-08-21,OTRO,CONCEPCION,SANTA JUANA,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,1.0,61402.0,1.3078,2.0,2.0,HOSPITAL CLORINDA AVELLO (SANTA JUANA),33.0,2019,3.0,2019-3
9,118100,1079955.0,HOMBRE,1944-12-07,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,14133.0,1.6687,3.0,3.0,NaN,31.0,2019,3.0,2019-3


Los procedimientos en esta base de datos se encuentran en las columnas "PROCEDIMIENTOSX", donde X
va desde 1 hasta 30. Para analizar esta base de forma más expedita, se transformará de formato
ancho a formato largo, utilizando como identificador a las columnas ["COD_HOSPITAL", "ANIO_EGRESO",
"CIP_ENCRIPTADO", "DIAGNOSTICO1"]. Cabe destacar que la columna "CIP_ENCRIPTADO" hace referencia al
RUT del paciente.


In [51]:
procedimientos_formato_long = pd.melt(
    df_procesada,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "TIPO_ACTIVIDAD", "CIP_ENCRIPTADO", "DIAGNOSTICO1"],
    value_vars=vars_procedimientos.keys(),
    value_name="procedimiento",
)

In [53]:
conteo_y_ranking_procedimientos = build_features.contar_procedimientos_y_rankear(
    procedimientos_formato_long
)

In [54]:
display(conteo_y_ranking_procedimientos.head(20))

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo,ranking,diferencia_proximo_hospital,total_procedimientos
180193,2021,113180,99.99,4,1,2,18
160738,2021,108100,99.99,2,2,1,18
148451,2021,103100,99.99,1,3,0,18
152391,2021,105100,99.99,1,4,0,18
166209,2021,110100,99.99,1,5,0,18
167455,2021,110120,99.99,1,6,0,18
179376,2021,113130,99.99,1,7,0,18
182902,2021,114101,99.99,1,8,0,18
187970,2021,115107,99.99,1,9,0,18
188758,2021,115110,99.99,1,10,0,18


El resultado muestra el ranking para todos los procedimientos asignados en el país en cada uno de 
los años analizados (2019 a 2021). 

A modo de ejemplo, se ilustra el ranking en el año 2021 para el procedimiento 99.99. En este caso,
se observa que los hospitales con código 108100 y 113180 están en los primeros lugares, realizando
2 procedimientos cada uno.

Ahora, si se filtra por el hospital del tórax, y sólo los procedimientos donde el hospital sea
ranking 1, entonces se obtendrán procedimientos altamente relevantes para el país.

In [12]:
cie_9 = pd.read_excel("../data/external/CIE-9.xlsx", dtype={"Código": str})
esquema_hospitales = pd.read_excel(
    "../data/external/Esquema_Registro-2023.xls", sheet_name=4, header=7
).dropna(how="all", axis=1)

In [13]:
ranking_con_nombre_procedimientos = pd.merge(
    conteo_y_ranking_procedimientos, cie_9, how="left", left_on="procedimiento", right_on="Código"
)
ranking_con_nombre_procedimientos_y_establecimientos = ranking_con_nombre_procedimientos.merge(
    esquema_hospitales, how="left", left_on="COD_HOSPITAL", right_on="Código nuevo Establecimiento"
)

In [14]:
display(ranking_con_nombre_procedimientos_y_establecimientos)

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo,ranking,diferencia_proximo_hospital,total_procedimientos,Código,Descripción,Categoría,...,Capítulo,Código nuevo Establecimiento,SEREMI / Servicio de Salud,Pertenencia al SNSS,Código tipo Pertenencia,TipoEstablecimiento,Código tipo establecimiento,Nombre Oficial,Código Comuna,Código Región
0,2021,113180,99.99,4,1,2,18,99.99,PROCEDIMIENTO MISCELANEO.OTRO,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,113180,Servicio de Salud Metropolitano Sur,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital El Pino (Santiago, San Bernardo)",13401,13
1,2021,108100,99.99,2,2,1,18,99.99,PROCEDIMIENTO MISCELANEO.OTRO,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,108100,Servicio de Salud Aconcagua,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Camilo de San Felipe,5701,5
2,2021,103100,99.99,1,3,0,18,99.99,PROCEDIMIENTO MISCELANEO.OTRO,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,103100,Servicio de Salud Antofagasta,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital Dr. Leonardo Guzmán (Antofagasta),2101,2
3,2021,105100,99.99,1,4,0,18,99.99,PROCEDIMIENTO MISCELANEO.OTRO,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,105100,Servicio de Salud Coquimbo,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Juan de Dios (La Serena),4101,4
4,2021,110100,99.99,1,5,0,18,99.99,PROCEDIMIENTO MISCELANEO.OTRO,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,110100,Servicio de Salud Metropolitano Occidente,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital San Juan de Dios (Santiago, Santiago)",13101,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214751,2019,105101,00.01,65,1,52,80,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,"00 PROCEDIMIENTOS E INTERVENCIONES, NO CLASIFI...",...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,105101,Servicio de Salud Coquimbo,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Pablo (Coquimbo),4102,4
214752,2019,106100,00.01,13,2,12,80,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,"00 PROCEDIMIENTOS E INTERVENCIONES, NO CLASIFI...",...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,106100,Servicio de Salud Valparaíso San Antonio,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital Carlos Van Buren (Valparaíso),5101,5
214753,2019,113100,00.01,1,3,0,80,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,"00 PROCEDIMIENTOS E INTERVENCIONES, NO CLASIFI...",...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,113100,Servicio de Salud Metropolitano Sur,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital Barros Luco Trudeau (Santiago, San Mi...",13130,13
214754,2019,120101,00.01,1,4,-1,80,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,"00 PROCEDIMIENTOS E INTERVENCIONES, NO CLASIFI...",...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,120101,Servicio de Salud Biobío,Perteneciente,1,Hospital (Alta Complejidad),4.0,Complejo Asistencial Dr. Víctor Ríos Ruiz (Los...,8301,8


Ahora se puede observar el nombre del procedimiento analizado y el nombre del hospital visto.

In [22]:
conteo_procedimientos_torax = ranking_con_nombre_procedimientos_y_establecimientos.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX"
).sort_values(["ANIO_EGRESO", "conteo"], ascending=False)

mejores_procedimientos_torax = conteo_procedimientos_torax.query("ranking == 1")

display(mejores_procedimientos_torax)

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo,ranking,diferencia_proximo_hospital,total_procedimientos,Código,Descripción,Categoría,...,Capítulo,Código nuevo Establecimiento,SEREMI / Servicio de Salud,Pertenencia al SNSS,Código tipo Pertenencia,TipoEstablecimiento,Código tipo establecimiento,Nombre Oficial,Código Comuna,Código Región
54295,2021,112103,39.61,626,1,68,3184,39.61,CIRCULACION EXTRACORPOREA PARA CIRUGIA CORAZON...,39 OTRAS OPERACIONES SOBRE VASOS,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
54251,2021,112103,39.64,584,1,246,2020,39.64,MARCAPASOS CORAZON INTRAOPERATORIO,39 OTRAS OPERACIONES SOBRE VASOS,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
52933,2021,112103,42.23,391,1,342,607,42.23,ESOFAGOSCOPIA OTRA,42 OPERACIONES SOBRE ESOFAGO,...,09. (42 54) OPERACIONES SOBRE EL APARATO DIGES...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
57293,2021,112103,37.22,280,1,22,801,37.22,CATETERISMO CORAZON LADO IZQUIERDO,37 OTRAS OPERACIONES SOBRE CORAZON Y PERICARDIO,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
13242,2021,112103,89.46,278,1,82,1355,89.46,COMPROBACION FORMA ONDA ARTEFACTO MARCAPASOS C...,"89 ENTREVISTA, EVALUACION CONSULTA Y EXAMEN",...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200802,2019,112103,34.89,2,1,1,6,34.89,OPERACION DIAFRAGMA.OTRA,"34 OPERA. SOBRE PARED TORACICA, PLEURA, MEDIAS...",...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
201716,2019,112103,33.49,2,1,1,10,33.49,OPERACION PULMON REPARADORA Y PLASTICA OTRA,33 OTRAS OPERACIONES SOBRE PULMON Y BRONQUIO,...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
211026,2019,112103,07.81,2,1,1,10,07.81,ESCISION TIMO PARCIAL OTRA,07 OPERACIONES SOBRE OTRAS GLANDULAS ENDOCRINAS,...,02. (06 07) OPERACIONES SOBRE EL SISTEMA ENDOC...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
198540,2019,112103,38.46,1,1,0,3,38.46,RESECCION ARTERIA ABDOMEN CON SUSTITUCION,"38 INCISION, EXCISION Y OCLUSION DE VASOS",...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13


Los resultados muestran que el tórax en el año 2021 fue altamente relevante en los procedimientos
39.61, 39.64, 42.23, entre otros. Además, se puede observar la diferencia que tiene el Tórax
con los siguientes establecimientos de salud y el total de procedimientos realizados en el país.

In [24]:
with pd.ExcelWriter("../data/interim/procedimientos_nacionales_ranking.xlsx") as file:
    ranking_con_nombre_procedimientos_y_establecimientos.to_excel(
        file, sheet_name="ranking_nacional", index=False
    )
    conteo_procedimientos_torax.to_excel(file, sheet_name="procedimientos_torax", index=False)
    mejores_procedimientos_torax.to_excel(
        file, sheet_name="procedimientos_torax_ranking_1", index=False
    )

In [42]:
df_procesada.TIPO_ACTIVIDAD.value_counts()

HOSPITALIZACIÓN                    2302880
CIRUGÍA MAYOR AMBULATORIA (CMA)     329516
HOSPITALIZACIÓN EN URGENCIA          62551
HOSPITALIZACIÓN DIURNA               55328
DESCONOCIDO                             18
NO IDENTIFICADO                          3
Name: TIPO_ACTIVIDAD, dtype: int64

In [50]:
df_procesada["USOSPABELLON"].value_counts().head(20)

1.0      876413
2.0      120819
3.0       49747
1         36598
4.0       10481
2          4884
5.0        3811
3          1700
0.0         166
6.0         161
4           115
8.0          59
200.0        27
10.0         23
227.0        14
245.0         8
11.0          8
12.0          8
9.0           8
209.0         6
Name: USOSPABELLON, dtype: int64